# Ontwikkeling positieve COVID19 testuitslagen
> RIVM data door de tijd

- toc: false
- branch: master
- badges: true
- comments: true
- author: Jesse van Elteren
- image: images/covidheader.png
- categories: []

Update 31 maart: Het RIVM is overgegaan op een ander type data, namelijk het aantal ziekenhuisopnames per gemeente, alleen zonder historie te publiceren. Hiermee komt er een einde aan deze visualisatie. Misschien ga ik de ziekenhuisopnames nog een keer visualiseren.

Update 18 april: uiteindelijk is de beste informatie de oversterfte per week. Deze is te vinden op [de site van het CBS](https://www.cbs.nl/nl-nl/dossier/cbs-cijfers-coronacrisis)

Dagelijks publiceert het RIVM de inmiddels welbekende testkaart met daarop het aantal mensen dat per 100.000 inwoners in de gemeente positief is getest op Covid-19.
Het leek me interessant om het verloop per gemeente te visualiseren.

Hieronder zie je drie grafieken:
- Aantal: Totaal aantal positieve testcases (cumulatief)
- Relatief: Totaal aantal positieve testcases per 100.000 inwoners
- Groei: Toename van het aantal gevallen ten opzichte van 4 dagen geleden. Factor 0 betekent geen extra gevallen, factor 1 betekent een verdubbeling en factor 2 betekent dat er 2x zoveel cases bij zijn gekomen als dat er 4 dagen geleden waren (een verdrievoudiging van het totaal aantal gevallen). 

In [1]:
# hide
import pandas as pd
from string import punctuation
import altair as alt

df = pd.read_csv('https://github.com/jvanelteren/covid19/raw/master/confirmed_Covid_19_NL.csv',dtype=object)
num_csv = int(max(df['Count']))+1
df[["Aantal", "Aantal per 100000 inwoners",'BevAant']] = df[["Aantal", "Aantal per 100000 inwoners",'BevAant']].apply(pd.to_numeric)

In [2]:
# hide
def clean_df(df):
    df = df.fillna(0)
    df = df.reindex(sorted(df.columns), axis=1)
    df.rename(columns=dict(zip(df.columns,[str(int(c)) if not c[0].isalpha() else c for c in df.columns])),inplace=True)
    return df

df_aantal = df.pivot(index='Gemeente', columns='Count', values='Aantal').reset_index()
df_aantal = clean_df(df_aantal)

df_relative = df.pivot(index='Gemeente', columns='Count', values='Aantal per 100000 inwoners').reset_index()
df_relative = clean_df(df_relative)

In [3]:
# hide
# make a growth df
df_growth = df_aantal
temp_columns = df_growth.columns
def calc_growth(line):
    alternative_growth = [round(y/x-1,2) if x>4 else 0 for x,y in zip(line[:-1],line[3:-1])]
    result = [0]*(len(line)-len(alternative_growth)-1) + alternative_growth + [line[-1]]
    return result
df_growth = pd.DataFrame.from_records(df_growth.apply(calc_growth,axis=1))
df_growth.rename(columns=dict(zip(df_growth.columns,temp_columns)),inplace=True)

In [4]:
# hide
url = 'https://cartomap.github.io/nl/wgs84/gemeente_2020.topojson' 
gemeentes = alt.topo_feature(url, 'gemeente_2020')

columns = [str(day) for day in range(0, num_csv)]
slider = alt.binding_range(min=0, max=num_csv-1, step=1)
select_year = alt.selection_single(name="select", fields=['day'],
                                   bind=slider, init={'day': num_csv-1})

def serve_chart(df, legend, legend_title):
    maximum = 3 if legend == 'groei' else max(df[str(num_csv-1)]) # max([max(df_growth[c]) for c in columns])
    chart = alt.Chart(gemeentes).mark_geoshape(
        stroke='black',
        strokeWidth=0.05
    ).transform_lookup(
        lookup='properties.statnaam',
        from_=alt.LookupData(df, 'Gemeente', columns),
        default = '0'
    ).transform_fold(
        columns, as_=['day', legend]
    ).transform_calculate(
        day='parseInt(datum.day)',
        legend='isValid(datum.'+legend+') ? datum.'+legend+' : -1'  
    ).encode(
        tooltip=[alt.Tooltip('properties.statnaam:N', title="Gemeente"), alt.Tooltip(legend+':Q', title=legend)],
        color = alt.condition(
            'datum.'+legend+' > 0',
            alt.Color(legend+':Q', scale=alt.Scale(scheme='spectral', type='symlog',domain=[0,maximum]),sort='descending',
                      legend=alt.Legend(orient='top',title=legend_title,gradientLength=330,tickCount=4, titleLimit=200)),
            alt.value('#dbe9f6')
        )).add_selection(
        select_year
    ).properties(
        width=330,
        height=400
    ).transform_filter(
        select_year).configure_view(strokeWidth=0)
    return chart

In [5]:
#hide_input
serve_chart(df_aantal, 'aantal', 'Positieve tests (cumulatief)')

alt.Chart(...)

Je kan met de slider het verloop van de epidemie volgen. Zoom in als je hem moeilijk te pakken krijgt. Dag 0 is 3 maart, dit was de eerste dag waarvan ik data kon terug vinden op de site van het RIVM. De laatste dag is 30 maart.

In [6]:
#hide_input
serve_chart(df_relative, 'relatief','Cases per 100.000 inwoners')

alt.Chart(...)

In [7]:
#hide_input
serve_chart(df_growth, 'groei', 'Groei tov 3 dagen eerder')

alt.Chart(...)

Tenslotte nog een mogelijkheid om het verloop in een specifieke gemeente te bekijken

In [8]:
#hide_input
test = df_aantal.T
test.columns = test.iloc[-1]
# test = test.iloc[:,4:-2]
test = test[:-1]
test = test.reset_index()
def change(col):
    return ''.join([c for c in col if c not in set(punctuation)-set(['-'])])
res = {c:change(c) for c in test.columns if set(punctuation) & set(c)-set(['-'])}
test.rename(columns=res, inplace=True)
test['Datum'] = list(df[['Datum','Count']].groupby('Count').describe()['Datum']['top'])
columns = list(test.columns)
to_remove = ['Count','Datum']
columns = [c for c in columns if len(c)<30 and c not in to_remove]
select_box = alt.binding_select(options=columns, name='Gemeente')
sel = alt.selection_single(fields=['Gemeente'], bind=select_box, init={'Gemeente': 'Amsterdam'})

alt.Chart(test).transform_fold(
    columns,
    as_=['Gemeente', 'value']
).transform_filter(
    sel  
).mark_line().encode(
    x=alt.X('Datum:O', title='Datum'),
    y=alt.X('value:Q', title='Total aantal positieve testen (cumulatief)'),
    tooltip=[alt.Tooltip('Datum:O', title="Datum"), alt.Tooltip('value:Q', title='Aantal')],
).add_selection(
    sel
).properties(
        width=325,
        height=325
).configure_view(strokeWidth=0)

alt.Chart(...)

> Important: Omdat in Nederland lang niet alle gevallen worden getest ligt het werkelijke aantal cases waarschijnlijk veel hoger. Weinig cases in jouw gemeente wil niet zeggen dat er niet meer gevallen zijn! Ook zal bijvoorbeeld een ziekenhuis in de gemeente de aantallen beinvloeden vanwege relatief vaker geteste zorgmedewerkers. Pas als er in Nederland veel meer gaat wordt getest kan je er echt conclusies aan verbinden. Landelijk zeggen [IC opnames](https://stichting-nice.nl/) momenteel meer.

> Warning: Je ziet dat sommige lijntjes heel erg stijl omhoog gaan, dat moeten we proberen te verminderen. Houdt je dus aan de maatregelen en adviezen.

> Note: Ik ben geen specialist en heb deze data van de RIVM site gedownload. Er kunnen dus ook geen rechten aan worden ontleend. De totalen zullen niet helemaal optellen tot de nationele aantallen omdat er ook gevallen zijn waarvan de woonplaats niet bekend is ten tijde van publicatie. Heb je tips laat maar weten!

> Tip: Bedankt ontwikkelaars van [Fastpages](https://fastpages.fast.ai/) (heel handige manier om Jupyter notebooks te bloggen) en [Altair](https://altair-viz.github.io/) (voor de visualisaties). Super om dit eens te proberen.